# First attempt

In [2]:
# Imports
import simpy
import random
import geopandas as gpd
from abc import ABC

random.seed(666)  # randomize with the beast

## Defining relevant classes

In [3]:
# TODO (@ps-jbunton): Find points where we reach across too many layers of inheritance and replace
# them with appropriate "getter" methods.  Biggest suspects: current_location.----.----, and all the
# location.name calls
class PayloadLocation(ABC):
    """
    Abstract base class for all possible locations that Payloads can be.
    """
    pass

class Payload:
    """
    Payload class.  Typically shipping containers, these represent objects that can be delivered via
    trucks or ParallelVehicles.

    
    Attributes
    ----------
    env: simpy.Environment
        The simpy simulation environment the Payload exists in.
    destination: PayloadLocation
        GeoLocation that the payload needs to be delivered.
    current_location: PayloadLocation
        The current location of the payload.
    origin: PayloadLocation
        Where the payload started its life.
    """

    def __init__(self, env: simpy.Environment, destination: PayloadLocation, current_location: PayloadLocation) -> None:
        """
        Constructs a new Payload object.

        Parameters
        ----------
        env: simpy.Environment
            The simpy simulation environment for the payload.
        destination: ContainerYard
            The payload's desired destination.
        current_location: ParallelVehicle | ContainerYard | RailTerminal
            Where to spawn the container.
        """
        self.env = env
        self.destination = destination
        self.current_location = current_location
        self.origin = current_location
    
    def __format__(self, spec) -> str:
        """
        Convenience method for printing out details of a Payload.
        """
        return f'Payload at {self.current_location}\nGoing to: {self.destination}'

In [4]:
class GeoLocation:
    """
    Physical location convenience class.

    Attributes
    ----------
    lat: float
        Latitude of the location.
    lon: float
        Longitude of the location.
    address: str | None
        String format of the address (for querying in Google Maps, for example)
    """
    def __init__(self, lat: float, lon: float, address: str | None = None) -> None:
        """
        Constructs a new GeoLocation.

        Parameters
        ----------
        lat: float
            GeoLocation's latitude coordinate.
        lon: float
            GeoLocation's longitude coordinate.
        address: str, optional
            String with street address of location.
        """
        self.lat = lat
        self.lon = lon
        self.address = address
    
    def __format__(self, spec) -> str:
        """
        Convenience method for printing out the details of a GeoLocation.
        """
        if self.address:
            return self.address
        else:
            return f'GeoLocation at ({self.lat:.6f}, {self.long:.6f})'

In [5]:
class RailTerminal(PayloadLocation):
    """
    Rail terminal class.  Models the railyard endpoints for routes, which may have
    `ParallelVehicle`s and `Truck`s enter it and move the `Payload`s through its
    `arrival_queue` and `departure_queue` simpy Stores.
    
    Attributes
    ----------
    env: simpy.Environment
        The simpy environment the RailTerminal exists in.
    location: GeoLocation | None
        The physical location to associate with this terminal, if any.
    arrival_queue: simpy.FilterStore
        A simpy Store object that holds Payloads that were brought to the terminal and are waiting
        to be loaded onto a rail vehicle.
    departure_queue: simpy.FilterStore
        A simpy Store object that holds Payloads brought to the terminal by rail vehicles that are
        waiting to be picked up by a road vehicle for delivery.
    name: str
        An identifying name for the rail terminal.
    """
    def __init__(self, env: simpy.Environment, name: str, location: GeoLocation | None = None) -> None:
        """
        Constructs a new rail terminal object.

        Parameters
        ----------
        env: simpy.Environment
            The simpy environment the rail terminal will exist in.
        name: str
            An identifying name for the rail terminal, i.e., "Savannah" or "Cordele"
        location: GeoLocation, optional
            An optional geographic location for the terminal to exist at.
        """
        self.env = env
        self.name = name
        self.location = location

        # Build the terminal's arrival and departure queues, use infinite capacity for now.
        self.arrival_queue = simpy.FilterStore(self.env)
        self.departure_queue = simpy.FilterStore(self.env)
    
    def __format__(self, spec) -> str:
        """
        Convenience function for writing out details of RailTerminal.
        """
        s = f'Name: {self.name}\n'
        if self.location:
            s += f'Location: {self.location}\n'
        s += f'arrival payload count: {len(self.arrival_queue.items)}\n'
        s += f'departure payload count: {len(self.departure_queue.items)}'
        return s

In [6]:
class ContainerYard(PayloadLocation):
    """
    Container yard class.  Models Container Yards, which request `Payload`s and receives them.

    Attributes
    ----------
    env: simpy.Environment
        The simpy environment the container yard lives in.
    id: int
        Unique int for identifying this ContainerYard from others.
    requests_per_day: float
        Average number of requests for Payloads that the container yard makes per day.  Requests
        will be generated using a Poisson process with rate parameter requests_per_day
    payload_sink: simpy.Container
        Simpy container that represents delivered payloads to this container yard.
    name: str
        Name of the container yard.
    location: GeoLocation | None
        Physical geographical location info for the container yard.
    """
    def __init__(self, env: simpy.Environment, requests_per_day: float, name: str, location: GeoLocation | None = None) -> None:
        """
        Creates a new ContainerYard.

        Parameters
        ----------
        env: simpy.Environment
            The simpy environment the new container yard will exist in.
        requests_per_day: float
            Average number of requests for new payloads this container yard makes per day.
        name: str
            Name of the container yard.
        location: GeoLocation, optional
            GeoLocation associated to this container yard, if any.
        """
        self.env = env
        self.requests_per_day = requests_per_day
        self.name = name
        self.location = location
        self.payload_sink = simpy.Container(self.env)
    
    def __format__(self, spec) -> str:
        return f'Name: {self.name}\nLocation: {self.location}\nNumber of received containers:{self.payload_sink.level}'
    
    def run(self, origin: PayloadLocation):
        """
        Convenience class for running the container yard logic.
        """
        while True:
            yield from self.new_order(origin)
    
    def new_order(self, origin: RailTerminal):
        """
        Function that generates a new Payload at the given origin whose destination is this
        ContainerYard at the rate self.requests_per_day.

        Parameters
        ----------
        origin: RailTerminal
            Where the newly created Payload instance should be generated.
        """
        # Wait for a randomly generated amount of time obeying a Poisson process.
        time_until_next_order_hrs = self._time_before_new_request_hrs()
        # print(f'T = {float(self.env.now):.03f} | Container yard {self.name} waiting {time_until_next_order_hrs} hours to generate a new order...')
        new_order = yield self.env.timeout(delay=time_until_next_order_hrs,
                                           value=Payload(self.env, self, origin),
                                           )
        # Put the new order in the wait queue of its origin.
        origin.departure_queue.put(new_order)
        print(f"T = {float(self.env.now):.03f} | Container yard {self.name} generated a new order at {new_order.origin.name}.")
    
    def _time_before_new_request_hrs(self):
        """
        Method that computes a new random time until the next request (in hours).  Assumes requests
        obey a Poisson arrival process, meaning they arrive with exponentially distributed 
        """
        return random.expovariate(self.requests_per_day / 24.0)

In [7]:
class ParallelVehicle(PayloadLocation):
    """
    Parallel vehicle class.  Models Parallel Vehicles as they move `Payload`s through the system.

    Attributes
    ----------
    env: simpy.Environment
        The simpy simulation environment the vehicle exists in.
    id: int
        Unique int for identifying this ParallelVehicle from others.
    capacity: int
        How many `Payload`s can be loaded on the vehicle.
    current_location: RailTerminal | None
        RailTerminal that the vehicle is currently at, or None if between.
    destination: RailTerminal | None
        The terminal the vehicle is headed to, or None if vehicle is not preparing to move.
    current_payload: List[Payload]
        List containing the Payload objects that the vehicle currently has loaded.
    """
    def __init__(self, env: simpy.Environment, id: int, capacity: int, current_location: RailTerminal, destination: RailTerminal | None = None, current_payload: list[Payload] | Payload | None = None) -> None:
        """
        Constructs a new ParallelVehicle.
        
        Parameters
        ----------
        env: simpy.Environment
            The simpy environment the vehicle will exist in.
        current_location: RailTerminal
            The RailTerminal to spawn this new vehicle in.
        capacity: int
            How many `Payload`s the vehicle can be loaded on this vehicle.
        destination: RailTerminal, optional
            The destination to give the vehicle, if any.
        current_payload: list[Payload] | Payload (optional)
            The Payload(s) to initialize the vehicle with, if any.
        """
        self.env = env
        self.current_location = current_location
        self.id = id
        self.capacity = capacity
        self.current_destination = destination
        self.current_payload = []
        if current_payload:
            if isinstance(current_payload, list):
                self.current_payload += current_payload
            elif isinstance(current_payload, Payload):
                self.current_payload.append(current_payload)
    
    def __format__(self, spec):
        """
        Convenience method for pretty-printing ParallelVehicle state.
        """
        s = f'Parallel Vehicle {self.id}\nAt location: {self.current_location.name}\n'
        if self.current_payload:
            s += f'Loaded with {len(self.current_payload)} payloads'
        else:
            s += f'Empty'
        if self.current_destination:
            s += f'\nGoing to: {self.current_destination.name}'
        return s

    
    def run(self):
        while True:
            # If there are things to be picked up here, go through the pickup procedure.
            if len(self.current_location.departure_queue.items) > 0:
                print(f'T = {float(self.env.now):.03f} | Parallel vehicle picking up from {self.current_location.name}')
                yield from self.pickup_payloads()
            
            # Travel between the terminals.
            print(f'T = {float(self.env.now):.03f} | Parallel vehicle leaving {self.current_location.name} with {len(self.current_payload)} payloads')
            previous_location = yield self.env.timeout(self._time_to_travel(), value=self.current_location)

            # Mark that the vehicle has arrived
            self.current_location = self.current_destination

            # For now, set the next destination as back where we came from.
            self.current_destination = previous_location
            print(f'T = {float(self.env.now):.03f} | Parllel vehicle arrived at {self.current_location.name}, dropping off.')
            yield from self.dropoff_payloads()
    
    def pickup_payloads(self):
        """
        Asks the vehicle to pick up new Payloads from its current RailTerminal until it is full.
        """
        while len(self.current_payload) < self.capacity:
            # while not at capacity, add Payloads to the vehicle.
            payload = yield self.current_location.departure_queue.get()
            self.current_payload.append(payload)
            print(f'T = {float(self.env.now):.03f} | Parallel Vehicle picked up a payload ({len(self.current_payload)}/{self.capacity})')
    
    def dropoff_payloads(self):
        """
        Asks the vehicle to drop off all of its current Payloads at its current RailTerminal.
        """
        while self.current_payload:
            # Removes payloads from vehicle in LIFO order.
            payload = self.current_payload.pop()
            # Set the current location of the payload to where the vehicle is.
            payload.current_location = self.current_location
            yield self.current_location.arrival_queue.put(payload)
            print(f'T = {float(self.env.now):.03f} | Parallel Vehicle dropped off a payload ({len(self.current_payload)}/{self.capacity})')
    
    def _time_to_travel(self):
        """
        Wrapper to report how long it takes the vehicle to travel between the RailTerminals.
        Right now, generates a random number, but in the future could be more complicated.
        """
        return random.uniform(5, 15)

In [8]:
class Truck(PayloadLocation):
    """
    Truck class.  Models trucks that can pick up and drop off containers from `ContainerYard`s and
    `RailTerminal`s, while traveling by road.

    Attributes
    ----------
    env: simpy.Environment
        The simpy environment the Truck will live in.
    id: int
        A unique int to help distinguish this `Truck` from others.
    capacity: int
        An int characterizing how many `Payload`s the truck can carry at once.
    current_payload: list[Payload]
        A list of the `Payload`s currently loaded onto this truck, or None if there aren't any.
    current_location: Terminal | ContainerYard
        The current location of the truck.
    current_destination: Terminal | ContainerYard | None
        Where the truck is headed next, or None if it is waiting.
    """
    def __init__(self, env: simpy.Environment, id: int, capacity: int, current_location: RailTerminal | ContainerYard, current_payload: list[Payload] | Payload | None = None, current_destination: RailTerminal | ContainerYard | None = None) -> None:
        self.env = env
        self.id = id
        self.current_location = current_location
        self.current_destination = current_destination
        self.capacity = capacity
        self.current_payload = []
        if current_payload:
            if isinstance(current_payload, list):
                self.current_payload += current_payload
            elif isinstance(current_payload, Payload):
                self.current_payload.append(current_payload)
    
    def __format__(self, spec) -> str:
        """
        Convenience method for printing out status of Truck.
        """
        s = f'Truck {self.id}\nAt location: {self.current_location.name}\n'
        if self.current_payload:
            s += f'Loaded with {len(self.current_payload)} payloads \n'
        else:
            s += f'Empty\n'
        if self.current_destination:
            s += f'Going to: {self.current_destination.name}'
        return s

    def run(self):
        """
        Runs the Truck logic.
        """
        while True:
            # Pickup any payloads from the current location if it has an arrival queue.
            if hasattr(self.current_location, 'arrival_queue'):
                yield from self.pickup_payloads()

            # Drive to the current destination (assumes we have one!)
            print(f'T = {float(self.env.now):.03f} | Truck {self.id} departing {self.current_location.name} for {self.current_destination.name}')
            yield from self.drive_to_destination()

            # Hold on to the previous location.
            previous_location = self.current_location

            # Mark that the vehicle has arrived.
            self.current_location = self.current_destination
            print(f'T = {float(self.env.now):.03f} | Truck {self.id} arrived at {self.current_location.name}')

            # Tell the truck to turn around after dropping its payloads (may be overwritten if we
            # pick up new payloads).
            self.current_destination = previous_location

            # Drop off payloads at current location (will only execute if we have payloads).
            if self.current_payload:
                yield from self.dropoff_payloads()


    def drive_to_destination(self):
        """
        Method that characterizes behavior during the driving sequence.  Right now it's just a
        randomized delay,  but in the future it could do something more complicated.
        """
        time_to_destination_hrs = self._time_to_destination_hrs()
        yield self.env.timeout(delay=time_to_destination_hrs, value=self.current_location)
    
    def dropoff_payloads(self):
        """
        Method describing behavior during the dropoff sequence at a ContainerYard.  Right now it
        just dumps its contents into the `ContainerYard`s `payload_sink`.
        """
        while self.current_payload:
            # Pull and deliver the payloads in LIFO order
            payload = self.current_payload.pop()
            yield self.current_location.payload_sink.put(1)
            print(f'T = {float(self.env.now):.03f} | Truck {self.id} dropped off a payload at {self.current_location.name} ({len(self.current_payload)}/{self.capacity})')
    
    def pickup_payloads(self):
        """
        Method describing behavior during the pickup sequence at a RailTerminal.  It starts by
        grabbing the first available container in the `RailTerminal`s `arrival_queue`.  If the
        `Truck`'s capacity is not met and there is another container in the
        `RailTerminal`'s `arrival_queue` with the same `destination` field, it pulls it.
        """
        # Pickup the first available payload.
        if len(self.current_payload) < self.capacity:
            payload = yield self.current_location.arrival_queue.get()
            print(f'T = {float(self.env.now):.03f} | Truck {self.id} picked up payload destined for {payload.destination.name} from {self.current_location.name}')
            self.current_destination = payload.destination
            self.current_payload.append(payload)
        
        # Lambda expression to filter out payloads with the same destination.
        destination_filter = lambda payload: payload.destination == self.current_destination

        while len(self.current_payload) < self.capacity and sum(destination_filter(payload) for payload in self.current_location.arrival_queue.items) > 0:
            payload = yield self.current_location.departure_queue.get(destination_filter)
            self.current_payload.append(payload)
            print(f'T = {float(self.env.now):.03f} | Truck {self.id} picked up a payload destined for {payload.destination} from {self.current_location.name}')

            

    def _time_to_destination_hrs(self):
        """
        Function that returns how long it takes to travel from the self.current_location to
        self.current_destination.  Right now, it's just a random number, but we could do something
        more complicated.
        """
        return random.uniform(0.5, 3.0)


# Running the simulation

In [31]:
# Create initial simpy Environment.
env = simpy.Environment()

In [32]:
# Create RailTerminal in Savannah.
savannah_location = GeoLocation(
    lat=32.076336,
    lon=-821.121959,
    address="1880 W Gwinnett St, Savannah, GA 31415",
)
savannah_terminal = RailTerminal(
    env=env,
    name="Savannah Marine Terminal",
    location=savannah_location,
)

# Create RailTerminal in Cordele.
cordele_location = GeoLocation(
    lat=31.968125,
    lon=-83.7420403,
    address="2902 E 13th Ave, Cordele, GA 31015",
)
cordele_terminal = RailTerminal(
    env=env,
    name="Cordele Intermodal Services, Inc.",
    location=cordele_location,
)

terminals = [savannah_terminal, cordele_terminal]

print("Created terminals!")
for t in terminals:
    print('='*50)
    print(f'{t}')
    print('='*50)

Created terminals!
Name: Savannah Marine Terminal
Location: 1880 W Gwinnett St, Savannah, GA 31415
arrival payload count: 0
departure payload count: 0
Name: Cordele Intermodal Services, Inc.
Location: 2902 E 13th Ave, Cordele, GA 31015
arrival payload count: 0
departure payload count: 0


In [33]:
# Create container yards using spreadsheet data from Marty and Mary.
container_yards_df = gpd.read_file('./data/Savannah to Cordele Intermodal Services50mi_nbrs_nodes.geojson')

REQUESTS_PER_DAY = 1.5

# Helper function for converting the geoJSON file to ContainerYard objects
def create_container_yard(container_yard: gpd.GeoSeries) -> ContainerYard:
    yard_location = GeoLocation(lat=container_yard[['LATITUDE']],
                                lon=container_yard[['LONGITUDE']],
                                address=container_yard[['Address']].item())
    return ContainerYard(env=env,
                        requests_per_day=REQUESTS_PER_DAY,
                        name=container_yard[['Corp Name']].item(),
                        location=yard_location)

container_yards = container_yards_df.apply(create_container_yard, axis=1).tolist()

print("Created container yards!")
for c in container_yards:
    print('='*50)
    print(f'{c}')
    print('='*50)
    

Created container yards!
Name: CORDELE INTERMODAL SERVICES
Location: P.O. BOX 876
Number of received containers:0
Name: XPO - OGLETHORPE
Location: 238 STAGECOACH ROAD
Number of received containers:0
Name: Blue Bird Corp
Location: 402 Bluebird Blvd
Number of received containers:0
Name: Perdue Farms
Location: 250 Georgia Highway 247 Spur
Number of received containers:0
Name: Orgill, Inc.
Location: 260 Jordan Rd
Number of received containers:0


In [34]:
# Create a set of trucks stationed at each container yard, leaving for Cordele.
TRUCK_CAPACITY = 1
trucks = []
for i, container_yard in enumerate(container_yards):
    trucks.append(Truck(env=env,
                        id=i,
                        capacity=TRUCK_CAPACITY,
                        current_location=container_yard,
                        current_payload=None,
                        current_destination=cordele_terminal))

# Check that we can print out and see the trucks.
print("Created trucks!")
for t in trucks:
    print('='*50)
    print(f'{t}')
    print('='*50)

Created trucks!
Truck 0
At location: CORDELE INTERMODAL SERVICES
Empty
Going to: Cordele Intermodal Services, Inc.
Truck 1
At location: XPO - OGLETHORPE
Empty
Going to: Cordele Intermodal Services, Inc.
Truck 2
At location: Blue Bird Corp
Empty
Going to: Cordele Intermodal Services, Inc.
Truck 3
At location: Perdue Farms
Empty
Going to: Cordele Intermodal Services, Inc.
Truck 4
At location: Orgill, Inc.
Empty
Going to: Cordele Intermodal Services, Inc.


In [35]:
# Create ParallelVehicle instances

# How many Parallel vehicles will we simulate?
NUM_PARALLEL_VEHICLES = 1

# How many payloads can each vehicle carry?
PARALLEL_VEHICLE_CAPACITY = 6

# Create all of the vehicles
parallel_vehicles = []
for i in range(NUM_PARALLEL_VEHICLES):
    parallel_vehicles.append(ParallelVehicle(env=env,
                                             id=i,
                                             capacity=PARALLEL_VEHICLE_CAPACITY,
                                             current_location=savannah_terminal,
                                             destination=cordele_terminal,
                                             current_payload=None))

print('Generated Parallel vehicles!')
for p in parallel_vehicles:
    print('='*50)
    print(f'{p}')
    print('='*50)

Generated Parallel vehicles!
Parallel Vehicle 0
At location: Savannah Marine Terminal
Empty
Going to: Cordele Intermodal Services, Inc.


In [36]:
# Add event generators for Payload request generation
for container_yard in container_yards:
    env.process(container_yard.run(savannah_terminal))

In [37]:
# Add event generators for Parallel vehicle actions
for parallel_vehicle in parallel_vehicles:
    env.process(parallel_vehicle.run())

In [38]:
# Add event generators for Truck actions
for truck in trucks:
    env.process(truck.run())

In [42]:
# Run that bad boy for some days!
DAYS_TO_RUN = 365
time_to_run_hrs = DAYS_TO_RUN*24

env.run(until=time_to_run_hrs)

ValueError: until (8760) must be greater than the current simulation time